# Oficina de dados abertos do Transferegov

O foco da oficina será na parte das emendas parlamentares chamadas "emendas Pix" que é operacionaliza pelo Transferegov, no módulo de Transferências Especiais. Os dados estão disponibilizados por meio de API de dados abertos.

Endereço da API: https://docs.api.transferegov.gestao.gov.br/transferenciasespeciais/#/plano_acao_especial/get_plano_acao_especial

## Plano de ação especial

Vamos carregar em uma única tabela todos os dados de Plano de Ação Especial.

### Baixar os dados

In [ ]:
import pandas as pd
import requests
from tqdm.notebook import tqdm

: 

In [ ]:
LIMIT = 1000
ANO = 2024
URL = "https://api.transferegov.gestao.gov.br/transferenciasespeciais/plano_acao_especial"

: 

In [ ]:
content = requests.get(
    URL,
    params={
        "ano_emenda_parlamentar_plano_acao": f"eq.{ANO}",
        "limit": 10,
    },
    headers={
        "Prefer": "count=exact",
    }
)

: 

In [ ]:
content.headers

: 

Vamos descobrir a quantidade de registros para iterar sobre ela.

In [ ]:
record_count = int(content.headers["Content-Range"].split("/")[-1])
record_count

: 

In [ ]:
data = content.json()

: 

In [ ]:
df = pd.DataFrame(columns=data[0].keys())

: 

In [ ]:
df.columns

: 

In [ ]:
pages = []
for offset in tqdm(range(0, record_count, LIMIT)):
    content = requests.get(
        URL,
        params={
            "ano_emenda_parlamentar_plano_acao": f"eq.{ANO}",
            "limit": LIMIT,
            "offset": offset,
        },
    )
    pages.extend(content.json())

df = pd.DataFrame(pages)
df

: 

### Análise

In [ ]:
df["nome_parlamentar_emenda_plano_acao"].value_counts()

: 

In [ ]:
df["valor_emenda"] = df["valor_custeio_plano_acao"] + df["valor_investimento_plano_acao"]

In [ ]:
total_emendas_por_parlamentar = df.groupby("nome_parlamentar_emenda_plano_acao").sum()["valor_emenda"]
total_emendas_por_parlamentar

nome_parlamentar_emenda_plano_acao
Abilio Brunini      8485793.0
Acácio Favacho     16732792.0
Adail Filho        18434456.0
Adilson Barroso    15515785.0
Adolfo Viana       15650000.0
                      ...    
Zé Silva           13835792.0
Zé Trovão          14935792.0
Zé Vitor           17671585.0
Átila Lins         15121585.0
Átila Lira         17585585.0
Name: valor_emenda, Length: 530, dtype: float64

In [ ]:
total_emendas_por_parlamentar.sort_values(ascending=False)[:20]

: 

In [ ]:
import matplotlib.pyplot as plt

# Ordenar os dados e selecionar os 20 maiores valores
top_20_emendas = total_emendas_por_parlamentar.sort_values(ascending=False)[:20]

# Criar o gráfico de barras horizontais
plt.figure(figsize=(12, 6))
top_20_emendas.plot(kind='barh', color='skyblue', edgecolor='black')

# Adicionar título e rótulos
plt.title('Top 20 Parlamentares por Valor de Emendas', fontsize=16)
plt.xlabel('Valor Total das Emendas', fontsize=12)
plt.ylabel('Parlamentares', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Ajustar layout e exibir
plt.tight_layout()
plt.show()


: 

In [ ]:


# URL da API da Câmara para obter partidos
url = "https://dadosabertos.camara.leg.br/api/v2/deputados"

# Requisição dos dados
response = requests.get(url, params={})
data = response.json()["dados"]

# Convertendo para DataFrame
#df_deputados = pd.DataFrame(data)[["id", "nome", "siglaPartido", "siglaUf"]]
df_deputados = pd.DataFrame(data)
df_deputados = df_deputados.rename(columns={"siglaPartido":"partido"})
# Exibindo os primeiros registros
print(df_deputados.head())


: 

In [ ]:
df.columns

: 

In [ ]:
#df.nome_parlamentar_emenda_plano_acao.unique()
#df_deputados.nome.unique()
#len(set(df.nome_parlamentar_emenda_plano_acao.unique())-set(df_deputados.nome.unique()))
#len(df_deputados.nome.unique())
#df_deputados[df_deputados['nome'].str.contains('Fernandes')]

: 

In [ ]:
import requests
import pandas as pd

# URL da API para buscar senadores
url = "https://legis.senado.leg.br/dadosabertos/senador/lista/atual.json"

# Fazer a requisição à API
response = requests.get(url)

# Verificar se a resposta foi bem-sucedida
if response.status_code == 200:
    # Parse do JSON
    data = response.json()

    # Extrair os dados relevantes
    senadores = data['ListaParlamentarEmExercicio']['Parlamentares']['Parlamentar']
    nomes = []
    partidos = []
    ufs = []

    for senador in senadores:
        nome = senador['IdentificacaoParlamentar']['NomeParlamentar']
        partido = senador['IdentificacaoParlamentar']['SiglaPartidoParlamentar']
        uf =  senador['IdentificacaoParlamentar']['UfParlamentar']
        nomes.append(nome)
        partidos.append(partido)
        ufs.append(uf)

    # Criar um DataFrame para exibir os dados
    #df = pd.DataFrame({'Nome': nomes, 'Partido': partidos})
    df_senadores = pd.DataFrame({'nome': nomes, 'partido': partidos, 'uf': ufs})
    print(df_senadores)
else:
    print(f"Erro ao acessar a API. Código de status: {response.status_code}")


: 

In [ ]:
df_deputados.columns

: 

In [ ]:

# Supondo que 'nome_parlamentar_emenda_plano_acao' seja a chave de união

# Merge com df_deputados
df_emendas = df.merge(
    df_deputados.rename(columns={'nome': 'nome_parlamentar_emenda_plano_acao'}),
    on='nome_parlamentar_emenda_plano_acao',
    how='left',
    suffixes=('', '_deputado')
)
df_emendas['cargo']=None
df_emendas.loc[df_emendas['partido'].notna(),['cargo']]='deputado'
print(df_emendas.columns)

# Merge com df_senadores
df_emendas = df_emendas.merge(
    df_senadores.rename(columns={'nome': 'nome_parlamentar_emenda_plano_acao'}),
    on='nome_parlamentar_emenda_plano_acao',
    how='left',
    suffixes=('', '_senador')
)
#
df_emendas.loc[df_emendas['partido_senador'].notna(),['cargo']]='senador'

# Verifica se as colunas existem antes de usar fillna
if 'partido_senador' in df_emendas.columns:
    df_emendas['partido'] = df_emendas['partido'].fillna(df_emendas['partido_senador'])
if 'uf_senador' in df_emendas.columns:
    df_emendas['uf'] = df_emendas['uf'].fillna(df_emendas['uf_senador'])

# Remover colunas extras se existirem
df_emendas = df_emendas.drop(
    columns=[col for col in ['partido_senador', 'uf_senador', 'partido_deputado', 'uf_deputado'] if col in df_emendas.columns]
)

# Resultado final
df_emendas


: 

In [ ]:
# Resultado final
#df_emendas.cargo.value_counts()
#df_emendas.partido

: 

In [ ]:
colunas = ['nome_parlamentar_emenda_plano_acao','partido']

: 

In [ ]:
print(deputados_df[colunas].isnull().sum())

In [ ]:
df_emendas_deputados = (
    df_emendas[df_emendas['cargo']=='deputado']
    .groupby(colunas)
    .sum()
    ['valor_emenda']
    .sort_values(ascending=False)
    .head(10)
)
df_emendas_deputados


In [ ]:
df_emendas_senadores = (
    df_emendas[df_emendas['cargo']=='senador']
    .groupby(colunas)
    .sum()
    ['valor_emenda']
    .sort_values(ascending=False)
    .head(10)
)
df_emendas_senadores

In [ ]:
# Criando o gráfico de barras horizontais
plt.figure(figsize=(10, 6))
plt.barh(df_emendas_senadores.index,
         df_emendas_senadores.values,
         color='skyblue')

# Adicionando título e rótulos aos eixos
plt.title('Top 10 Senadores por Valor de Emendas', fontsize=16)
plt.xlabel('Valor Total das Emendas', fontsize=12)
plt.ylabel('Senadores', fontsize=12)

# Exibindo os valores nas barras
for index, value in enumerate(df_emendas_senadores.values):
    plt.text(value, index, f'{value:,.2f}', va='center', fontsize=10)

# Ajustando o layout
plt.tight_layout()

# Exibindo o gráfico
plt.show()

In [ ]:
# Criando o gráfico de barras horizontais
plt.figure(figsize=(10, 6))
plt.barh(df_emendas_deputados.index,
         df_emendas_deputados.values,
         color='skyblue')

# Adicionando título e rótulos aos eixos
plt.title('Top 10 Deputados por Valor de Emendas', fontsize=16)
plt.xlabel('Valor Total das Emendas', fontsize=12)
plt.ylabel('Deputados', fontsize=12)

# Exibindo os valores nas barras
for index, value in enumerate(df_emendas_senadores.values):
    plt.text(value, index, f'{value:,.2f}', va='center', fontsize=10)

# Ajustando o layout
plt.tight_layout()

# Exibindo o gráfico
plt.show()